In [ ]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline  

import plotly.graph_objects as go

tfk = tf.keras
tfkl = tfk.layers

In [ ]:
def helmholtz(x, y, a_1, a_2):
    x = 2*x 
    y = 2*y
    z = np.sin(a_1 * np.pi * x) * np.sin(a_2 * np.pi * y)
    
    # Add noise to z
    noise = np.random.normal(0, 0.1, z.shape)
    z += noise
    
    return z

In [ ]:
A_1, A_2 = (1, 4)

In [ ]:
X, Y = np.meshgrid(
    np.linspace(-1, 1, 100), 
    np.linspace(-1, 1, 100)
)
Z = helmholtz(X, Y, A_1, A_2)

fig = go.Figure(data=[
    go.Surface(
        z=Z, 
        x=X, 
        y=Y, 
    )
])

fig.update_traces(contours_z=dict(show=True, usecolormap=True, highlightcolor="limegreen", project_z=True))

fig.update_layout(
    title='Original function', autosize=True,
    width=512, 
    height=512,
)

fig.show()

In [ ]:
from skopt.space import Space
from skopt.sampler import Halton


n_samples = 150*150
space = Space([(-1.0, 1.0), (-1.0, 1.0)])

sampler = Halton()

x_train = np.array(sampler.generate(space.dimensions, n_samples))
y_train = helmholtz(x_train[:,0], x_train[:,1], A_1, A_2).reshape((-1, 1))

In [ ]:
from arnold.layers.wavelet import (
    Bump, 
    Meyer,
    Morelet,
    Poisson,
    Ricker, 
    Shannon
    
)

from arnold.layers.radial import (
    GaussianRBF,
    InverseQuadraticRBF,
    InverseMultiQuadricRBF
)

In [ ]:
all_models = {
    'mlp': tfk.Sequential([
        tfkl.Dense(1024, activation="relu"),
        tfkl.Dense(512, activation="relu"),
        tfkl.Dense(1, activation="linear")
        ],
        name='mlp'
    ),
    'bump': tfk.Sequential([
            Bump(input_dim=2, output_dim=8),
            tfkl.LayerNormalization(),
            Bump(input_dim=8, output_dim=16),
            tfkl.LayerNormalization(),
            Bump(input_dim=16, output_dim=32),
            tfkl.LayerNormalization(),
            Bump(input_dim=32, output_dim=1),
        ],
        name='bump_kan'
    ),
    'meyer': tfk.Sequential([
            Meyer(input_dim=2, output_dim=8),
            tfkl.LayerNormalization(),
            Meyer(input_dim=8, output_dim=16),
            tfkl.LayerNormalization(),
            Meyer(input_dim=16, output_dim=32),
            tfkl.LayerNormalization(),
            Meyer(input_dim=32, output_dim=1),
        ],
        name='meyer_kan'
    ),
    'morelet': tfk.Sequential([
            Morelet(input_dim=2, output_dim=8),
            tfkl.LayerNormalization(),
            Morelet(input_dim=8, output_dim=16),
            tfkl.LayerNormalization(),
            Morelet(input_dim=16, output_dim=32),
            tfkl.LayerNormalization(),
            Morelet(input_dim=32, output_dim=1),
        ],
        name='morelet_kan'
    ),
    'poisson': tfk.Sequential([
            Poisson(input_dim=2, output_dim=8),
            tfkl.LayerNormalization(),
            Poisson(input_dim=8, output_dim=16),
            tfkl.LayerNormalization(),
            Poisson(input_dim=16, output_dim=32),
            tfkl.LayerNormalization(),
            Poisson(input_dim=32, output_dim=1),
        ],
        name='poisson_kan'
    ),
    'ricker': tfk.Sequential([
            Ricker(input_dim=2, output_dim=8),
            tfkl.LayerNormalization(),
            Ricker(input_dim=8, output_dim=16),
            tfkl.LayerNormalization(),
            Ricker(input_dim=16, output_dim=32),
            tfkl.LayerNormalization(),
            Ricker(input_dim=32, output_dim=1),
        ],
        name='ricker_kan'
    ),
    'shannon': tfk.Sequential([
            Shannon(input_dim=2, output_dim=8),
            tfkl.LayerNormalization(),
            Shannon(input_dim=8, output_dim=16),
            tfkl.LayerNormalization(),
            Shannon(input_dim=16, output_dim=32),
            tfkl.LayerNormalization(),
            Shannon(input_dim=32, output_dim=1),
        ],
        name='shannon_kan'
    ),

    'gaussian_rbf': tfk.Sequential([
            GaussianRBF(input_dim=2, output_dim=8),
            tfkl.LayerNormalization(),
            GaussianRBF(input_dim=8, output_dim=16),
            tfkl.LayerNormalization(),
            GaussianRBF(input_dim=16, output_dim=32),
            tfkl.LayerNormalization(),
            GaussianRBF(input_dim=32, output_dim=1),
        ],
        name='gaussian_rbf_kan'
    ),
    'inverse_quadratic_rbf': tfk.Sequential([
            InverseQuadraticRBF(input_dim=2, output_dim=8),
            tfkl.LayerNormalization(),
            InverseQuadraticRBF(input_dim=8, output_dim=16),
            tfkl.LayerNormalization(),
            InverseQuadraticRBF(input_dim=16, output_dim=32),
            tfkl.LayerNormalization(),
            InverseQuadraticRBF(input_dim=32, output_dim=1),
        ],
        name='inverse_quadratic_rbf_kan'
    ),
    'inverse_multiquadric_rbf': tfk.Sequential([
            InverseMultiQuadricRBF(input_dim=2, output_dim=8),
            tfkl.LayerNormalization(),
            InverseMultiQuadricRBF(input_dim=8, output_dim=16),
            tfkl.LayerNormalization(),
            InverseMultiQuadricRBF(input_dim=16, output_dim=32),
            tfkl.LayerNormalization(),
            InverseMultiQuadricRBF(input_dim=32, output_dim=1),
        ],
        name='inverse_multiquadric_rbf_kan'
    ),
}

In [ ]:
for name, model in tqdm(all_models.items()):
    model.build((None, 2))
    model.compile(
        optimizer=tf.keras.optimizers.Nadam(),
        loss='huber',
        metrics=['mse']
    )

print('Trainable parameter', {name: np.sum([np.prod(p.shape) for p in model.trainable_weights]) for (name, model) in all_models.items()})
print('Non-trainable parameter', {name: np.sum([np.prod(p.shape) for p in model.non_trainable_weights]) for (name, model) in all_models.items()})

In [ ]:
EPOCHS = 100
BATCH_SIZE = 512

In [ ]:
model_train_histories = {
    name: model.fit(
        x_train,
        y_train,
        epochs=EPOCHS, 
        batch_size=BATCH_SIZE,
        shuffle=True,
        verbose=0
    ) for (name, model) in all_models.items()
}

In [ ]:
import pandas as pd

for name, hist in model_train_histories.items():
    pd.DataFrame(hist.history).plot(figsize=(8,5), title=name)
    plt.show()

In [ ]:
X_test, Y_test = np.meshgrid(
    np.linspace(-1, 1, 400), 
    np.linspace(-1, 1, 400)
)

x_test = np.stack([X_test.ravel(), Y_test.ravel()], axis=-1)
y_true = helmholtz(X_test, Y_test, A_1, A_2)

In [ ]:
all_predictions = { 
    name: model.predict(x_test).reshape((400,400)) for (name, model) in tqdm(all_models.items())
}

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, 
    cols=int((len(all_predictions) + 1)/2), 
    start_cell="top-left", 
    subplot_titles=['original', ] + list(all_predictions.keys()),
    specs=[[{"type": "surface"}, {"type": "surface"}, {"type": "surface"}, {"type": "surface"}, {"type": "surface"}], [{"type": "surface"}, {"type": "surface"}, {"type": "surface"}, {"type": "surface"}, {"type": "surface"},]]
)

fig.add_trace(
    go.Surface(
        z=Z, 
        y=X, 
        x=Y, 
    ),
    row=1, col=1
)

for i, (name, y) in enumerate(all_predictions.items()):
    fig.add_trace(
        go.Surface(
            z=y, 
            x=X_test, 
            y=Y_test, 
        ),
        row=1 + int(i / 5) if i < 4 else max(i/5, 2), 
        col=i+2 if i<4 else 1+(1 + i%5)%5
    )

fig.update_traces(contours_z=dict(show=True, usecolormap=True, highlightcolor="limegreen", project_z=True))
fig.update_layout(scene=dict(zaxis=dict(dtick=1, type='linear')))

fig.update_layout(
    autosize=True,
    width=2048, 
    height=1024,
)